In [1]:
import pandas as pd
import os

In [2]:
FILE_PATH = os.path.join(os.path.abspath(''), '..', 'raw_data', 'population_data_2010-2019.csv')
raw_teens = pd.read_csv(FILE_PATH, engine='python')

FILE_PATH = os.path.join(os.path.abspath(''), '..', 'raw_data', 'population_data_2000-2010.csv')
raw_aughts = pd.read_csv(FILE_PATH, engine='python')


In [3]:
teens = raw_teens[['NAME', 'STNAME', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019']]
teens.columns = ['name', 'state', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
teens = teens.drop_duplicates()

In [4]:
aughts = raw_aughts[['NAME', 'STNAME', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009']]
aughts.columns = ['name', 'state', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009']
aughts = aughts.drop_duplicates()

In [5]:
merged = teens.merge(aughts, on=['name', 'state'], how='left')
merged

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,...,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
1,Abbeville city,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,...,2985.0,2941.0,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0
2,Adamsville city,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,...,5021.0,4960.0,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0
3,Addison town,Alabama,751,750,743,742,739,734,731,726,...,701.0,701.0,708.0,714.0,722.0,729.0,741.0,750.0,752.0,759.0
4,Akron town,Alabama,355,347,347,343,338,339,333,332,...,485.0,473.0,454.0,442.0,426.0,416.0,404.0,395.0,384.0,369.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99274,Balance of Uinta County,Wyoming,4813,4793,4827,4828,4807,4807,4812,4762,...,4561.0,4497.0,4531.0,4501.0,4493.0,4493.0,4537.0,4638.0,4734.0,4830.0
99275,Washakie County,Wyoming,8530,8449,8409,8413,8273,8278,8165,8010,...,8252.0,8068.0,7988.0,7976.0,7960.0,8022.0,7979.0,8169.0,8229.0,8423.0
99276,Balance of Washakie County,Wyoming,2782,2755,2734,2737,2690,2690,2650,2599,...,2539.0,2500.0,2490.0,2502.0,2514.0,2548.0,2550.0,2626.0,2660.0,2739.0
99277,Weston County,Wyoming,7198,7142,7077,7136,7138,7208,7220,6968,...,6627.0,6487.0,6578.0,6610.0,6646.0,6594.0,6717.0,7033.0,7133.0,7266.0


In [6]:
# creating a feature that will tell if entry is a city, state or county
merged['type'] = merged['name']

# removing all of the county balances that are not part of any city
merged['type'] = merged['type'].apply(lambda x: 'balance' if x.startswith('Balance of') else x)
merged = merged[merged['type'] != 'balance']

In [7]:
# seperating out the counties
merged['type'] = merged['type'].apply(lambda x: 'county' if x.endswith('County') else x)
counties = merged[merged['type'] == 'county']
counties = counties.drop('type', axis=1)
merged = merged[merged['type'] != 'county']
counties

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
463,Autauga County,Alabama,54773,55227,54954,54727,54893,54864,55243,55390,...,44021.0,44889.0,45909.0,46800.0,48366.0,49676.0,51328.0,52405.0,53277.0,54135.0
469,Baldwin County,Alabama,183112,186558,190145,194885,199183,202939,207601,212521,...,141342.0,144875.0,147957.0,151509.0,156266.0,162183.0,168121.0,172404.0,175827.0,179406.0
471,Barbour County,Alabama,27327,27341,27169,26937,26755,26283,25806,25157,...,29015.0,28863.0,28653.0,28594.0,28287.0,28027.0,27861.0,27757.0,27808.0,27657.0
473,Bibb County,Alabama,22870,22745,22667,22521,22553,22566,22586,22550,...,19913.0,21028.0,21199.0,21399.0,21721.0,22042.0,22099.0,22438.0,22705.0,22941.0
479,Blount County,Alabama,57376,57560,57580,57619,57526,57526,57494,57787,...,51107.0,51845.0,52551.0,53457.0,54124.0,54624.0,55485.0,56240.0,57055.0,57341.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99269,Sweetwater County,Wyoming,43574,43986,45002,45157,44948,44719,44222,43464,...,37552.0,36899.0,37428.0,37450.0,38026.0,38739.0,39749.0,41470.0,42358.0,44133.0
99271,Teton County,Wyoming,21296,21414,21624,22315,22773,23047,23234,23384,...,18381.0,18653.0,18837.0,19066.0,19467.0,19632.0,20014.0,20472.0,20988.0,21232.0
99273,Uinta County,Wyoming,21089,20896,20996,20951,20822,20763,20682,20431,...,19666.0,19413.0,19587.0,19480.0,19470.0,19494.0,19709.0,20171.0,20613.0,21054.0
99275,Washakie County,Wyoming,8530,8449,8409,8413,8273,8278,8165,8010,...,8252.0,8068.0,7988.0,7976.0,7960.0,8022.0,7979.0,8169.0,8229.0,8423.0


In [8]:
# seperating out the states
states_list = set(merged['state'].to_list())
merged['type'] = merged['type'].apply(lambda x: 'state' if x in states_list else x)
states = merged[merged['type'] == 'state']
states = states.drop('type', axis=1)
merged = merged[merged['type'] != 'state']
states

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,...,4452173.0,4467634.0,4480089.0,4503491.0,4530729.0,4569805.0,4628981.0,4672840.0,4718206.0,4757938.0
787,Alaska,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,...,627963.0,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0
987,Arizona,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,...,5160586.0,5273477.0,5396255.0,5510364.0,5652404.0,5839077.0,6029141.0,6167681.0,6280362.0,6343154.0
1141,Arkansas,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,...,2678588.0,2691571.0,2705927.0,2724816.0,2749686.0,2781097.0,2821761.0,2848650.0,2874554.0,2896843.0
1874,California,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,...,33987977.0,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0
2469,Colorado,Colorado,5047349,5121108,5192647,5269035,5350101,5450623,5539215,5611885,...,4326921.0,4425687.0,4490406.0,4528732.0,4575013.0,4631888.0,4720423.0,4803868.0,4889730.0,4972195.0
2960,Connecticut,Connecticut,3579114,3588283,3594547,3594841,3594524,3587122,3578141,3573297,...,3411777.0,3432835.0,3458749.0,3484336.0,3496094.0,3506956.0,3517460.0,3527270.0,3545579.0,3561807.0
3183,Delaware,Delaware,899593,907381,915179,923576,932487,941252,948921,956823,...,786373.0,795699.0,806169.0,818003.0,830803.0,845150.0,859268.0,871749.0,883874.0,891730.0
3259,District of Columbia,District of Columbia,605226,619800,634924,650581,662328,675400,685815,694906,...,572046.0,574504.0,573158.0,568502.0,567754.0,567136.0,570681.0,574404.0,580236.0,592228.0
3261,Florida,Florida,18845537,19053237,19297822,19545621,19845911,20209042,20613477,20963613,...,16047515.0,16356966.0,16689370.0,17004085.0,17415318.0,17842038.0,18166990.0,18367842.0,18527305.0,18652644.0


In [9]:
# seperating out the cities
types_that_are_cities = ['city', 'town', 'village']
merged['type'] = merged['name']
merged['type'] = merged['type'].apply(lambda x: 'city' if x.split()[-1] in types_that_are_cities else x)
cities = merged[merged['type'] == 'city']
cities = cities.drop('type', axis=1)
cities['name'] = cities['name'].apply(lambda x: x.replace(' city', ''))
cities['name'] = cities['name'].apply(lambda x: x.replace(' town', ''))
cities['name'] = cities['name'].apply(lambda x: x.replace(' village', ''))
cities

,name,state,2010,2011,2012,2013,2014,2015,2016,2017,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
1,Abbeville,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,...,2985.0,2941.0,2909.0,2882.0,2857.0,2820.0,2807.0,2784.0,2742.0,2714.0
2,Adamsville,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,...,5021.0,4960.0,4894.0,4841.0,4784.0,4728.0,4687.0,4633.0,4594.0,4558.0
3,Addison,Alabama,751,750,743,742,739,734,731,726,...,701.0,701.0,708.0,714.0,722.0,729.0,741.0,750.0,752.0,759.0
4,Akron,Alabama,355,347,347,343,338,339,333,332,...,485.0,473.0,454.0,442.0,426.0,416.0,404.0,395.0,384.0,369.0
5,Alabaster,Alabama,31209,31375,31684,31980,32182,32772,33017,33275,...,24116.0,24674.0,25223.0,25957.0,26739.0,27504.0,28462.0,29020.0,29630.0,30107.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99224,Wamsutter,Wyoming,450,453,462,487,508,499,493,483,...,263.0,273.0,290.0,305.0,322.0,341.0,364.0,392.0,414.0,444.0
99225,Wheatland,Wyoming,3622,3620,3626,3622,3642,3645,3587,3549,...,3542.0,3559.0,3568.0,3548.0,3576.0,3568.0,3584.0,3594.0,3584.0,3592.0
99226,Worland,Wyoming,5487,5436,5419,5419,5328,5332,5263,5158,...,5363.0,5237.0,5180.0,5166.0,5149.0,5184.0,5150.0,5268.0,5301.0,5420.0
99227,Wright,Wyoming,1810,1812,1864,1860,1856,1885,1857,1760,...,1362.0,1387.0,1443.0,1454.0,1463.0,1498.0,1558.0,1640.0,1684.0,1791.0
